In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import scipy
import os
from datetime import datetime
import xgboost
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import mlflow
import warnings

warnings.filterwarnings("ignore")

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2023/11/18 11:27:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/11/18 11:27:46 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1', creation_time=1700324867126, experiment_id='1', last_update_time=1700324867126, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

### Functions

In [3]:
def load_pickle(filename:str):
    """this functions loads a pickle file
    Args:
        filename (str): path to the pickle file
    Returns:
        object: the object contained in the pickle file"""
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [4]:
X_train, y_train = load_pickle(os.path.join("data/data_processed", "train.pkl"))
X_val, y_val = load_pickle(os.path.join("data/data_processed", "valid.pkl"))
X_test, y_test = load_pickle(os.path.join("data/data_processed", "test.pkl"))

In [7]:
def run_lasso_regression(X_train: scipy.sparse._csr.csr_matrix, y_train: scipy.sparse._csr.csr_matrix, 
                         X_val: scipy.sparse._csr.csr_matrix, y_val: scipy.sparse._csr.csr_matrix, 
                         X_test: scipy.sparse._csr.csr_matrix, y_test: scipy.sparse._csr.csr_matrix):
    """
    
    Perform Lasso regression using the provided training, validation, and test data, 
    remember we are using as a backend store uri mlflow.db
    copy this in the terminal mlflow ui --backend-store-uri sqlite:///mlflow.db
    
    Parameters:
    - X_train: Training feature data
    - y_train: Training target data
    - X_val: Validation feature data
    - y_val: Validation target data
    - X_test: Test feature data
    - y_test: Test target data

    Returns:
    - Trained Lasso regression model
    """

    with mlflow.start_run(run_name="lasso_regression"):
        mlflow.set_tag("developer", "Maria")
        mlflow.set_tag("model", "lasso_regression")
        alpha = 0.1
        mlflow.log_param("alpha", alpha)
        
        lasso_reg = Lasso(alpha)
        lasso_reg.fit(X_train, y_train)

        y_pred_val = lasso_reg.predict(X_val)
        rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)
        mlflow.log_metric("rmse_validation", rmse_val)

        y_pred_test = lasso_reg.predict(X_test)
        rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
        mlflow.log_metric("rmse_test", rmse_test)

        # Log the trained model as an artifact
        mlflow.sklearn.log_model(lasso_reg, "models/lasso_regression_model")
        mlflow.log_artifact("data/data_raw", "data")
        mlflow.log_artifact("data/data_processed", "data")

        return lasso_reg


In [8]:
run_lasso_regression(X_train, y_train, X_val, y_val, X_test, y_test)

Lasso(alpha=0.1)

Validación de hiperparametros con optimización del rmse con Hyperot. 

In [9]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [10]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)
test = xgb.DMatrix(X_test, label=y_test)


In [11]:
def objective(params: dict):
    """
    Optimize an XGBoost model using hyperopt and MLflow.
    Parameters:
    - params (dict): Dictionary containing XGBoost hyperparameters.
    Returns:
    - dict: Dictionary containing loss and status for the hyperparameter optimization.
    """
    with mlflow.start_run(run_name="xgboost"):   
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("developer", "Maria")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation'), (test, 'test')],  
            early_stopping_rounds=15
        )
        y_pred_valid = booster.predict(valid)
        rmse_valid = mean_squared_error(y_val, y_pred_valid, squared=False)
        mlflow.log_metric("rmse_valid", rmse_valid) 

        y_pred_test = booster.predict(test)
        rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
        mlflow.log_metric("rmse_test", rmse_test) 

        # Log the trained model
        mlflow.sklearn.log_model(booster, "xgboost_model")
        return {'loss': rmse_valid, 'status': STATUS_OK}

In [12]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:8.14250	test-rmse:8.42157         
[1]	validation-rmse:7.27536	test-rmse:7.56531         
[2]	validation-rmse:6.64415	test-rmse:6.94042         
[3]	validation-rmse:6.19436	test-rmse:6.49168         
[4]	validation-rmse:5.88430	test-rmse:6.17891         
[5]	validation-rmse:5.66529	test-rmse:5.95568         
[6]	validation-rmse:5.51722	test-rmse:5.80235         
[7]	validation-rmse:5.41258	test-rmse:5.69092         
[8]	validation-rmse:5.34438	test-rmse:5.61557         
[9]	validation-rmse:5.29575	test-rmse:5.56138         
[10]	validation-rmse:5.26354	test-rmse:5.52175        
[11]	validation-rmse:5.23608	test-rmse:5.49043        
[12]	validation-rmse:5.21670	test-rmse:5.46722        
[13]	validation-rmse:5.20624	test-rmse:5.45341        
[14]	validation-rmse:5.19439	test-rmse:5.43911        
[15]	validation-rmse:5.18658	test-rmse:5.42792        
[16]	validation-rmse:5.17833	test-rmse:5.41732        
[17]	validation-rmse:5.17469	test-rmse:5.41087        
[18]	valid

Vamos a la UI y simulemos que algún cientifico de datos o miembro del equipo decide registrar ciertos podemos en "staging", y más adelante nosotros como ingenieros de ML tomamos la decisión de cual modelo registrar para producción bajo las métricas de performance obtenidas y las pruebas. Es muy importante, hacer las pruebas de validación antes de considerar un modelo en producción. Entre ellas se encuentran:

#### 1. Pruebas de Funcionalidad:

* Funcionalidad del Modelo: Asegurarse de que el modelo funcione correctamente en el entorno de producción con los datos reales (Se puede simular con cierta parte de la data o toda idealmente).
* Integración del Modelo: Verificar que el modelo integrado en la infraestructura de producción responda y se comunique adecuadamente con otros componentes del sistema (partes del Pipeline).

####  2. Pruebas de Rendimiento:
* Rendimiento del Modelo: Evaluar el rendimiento del modelo en términos de velocidad de inferencia, uso de recursos (CPU, memoria), y latencia para garantizar que cumple con los requisitos de producción. (El tiempo es un factor muy importante en la generación de soluciones de negocio)
* Escala y Carga: Probar el modelo bajo cargas esperadas para asegurar que el sistema es escalable y puede manejar la demanda prevista. (Orar para que el Kernel no muera con la carga, de lo contrario mirar formas de optimizar el código, hacer carga de datos por chuncks, paralelizar procesos, etc)

#### 3. Pruebas de Robustez:
* Robustez de estrés: Evaluar el comportamiento del modelo en condiciones adversas, como datos de entrada no esperados, valores faltantes, valores atípicos, etc. (Esto es muy importante, ya que en la vida real los datos no son perfectos, y es muy probable que el modelo se caiga si no se tiene en cuenta este factor por lo que generar pruebas unitarias para este tipo de casos es muy importante). 


### Making predictions from mlflow artifacts

Predict on Pandas DataFrame: 

In [13]:
logged_model = 'runs:/245ccb12106e4ea5b990d7bb1f706a1f/xgboost_model'
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [14]:
# get info from the loaded model
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: xgboost_model
  flavor: mlflow.sklearn
  run_id: 245ccb12106e4ea5b990d7bb1f706a1f

In [15]:
xgboost_model = mlflow.sklearn.load_model(logged_model)

In [16]:
xgboost_model.predict(valid)

array([20.630875, 21.042545, 32.280148, ..., 30.592302, 21.187845,
       16.8023  ], dtype=float32)

In [17]:
valid.get_label()

array([19.583334, 17.55    , 23.716667, ..., 17.      , 17.      ,
        5.      ], dtype=float32)

## Model registry and state transitions

<p align="center">
  <img src="https://pbs.twimg.com/media/EoOBoWyWEAAA8In.jpg" width="400" style="display: block; margin: auto;">
</p>


* Tener en cuenta el tiempo de ejecución

* Métricas obtenidas

* Size del tamaño 

* Las pruebas que anteriormente mencionamos. 


In [19]:
from mlflow.tracking import MlflowClient


In [20]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [21]:
runs = mlflow.search_runs()
# Extrae los IDs únicos de los experimentos
experiment_ids = runs['experiment_id'].unique()

In [22]:
experiment_ids

array(['1'], dtype=object)

In [23]:
client.create_experiment("regression_models")

'2'

In [24]:
runs = client.search_runs(experiment_ids=["1"])
for run in runs:
    print(run.info)


<RunInfo: artifact_uri='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1/3ff51bc075054d90b4da843ec170ec3c/artifacts', end_time=1700325662449, experiment_id='1', lifecycle_stage='active', run_id='3ff51bc075054d90b4da843ec170ec3c', run_name='xgboost', run_uuid='3ff51bc075054d90b4da843ec170ec3c', start_time=1700325650952, status='FINISHED', user_id='mdurango'>
<RunInfo: artifact_uri='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1/400b762199204811bef7760b30f65364/artifacts', end_time=1700325650936, experiment_id='1', lifecycle_stage='active', run_id='400b762199204811bef7760b30f65364', run_name='xgboost', run_uuid='400b762199204811bef7760b30f65364', start_time=1700325632407, status='FINISHED', user_id='mdurango'>
<RunInfo: artifact_uri='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1/8197c7d8b36940d4b49cb2a4065c3cfd/artifacts', end_time=1700325632391, experiment_id='1', lifecycle_stage='active', run_id='8197c7d8b36940d4b49cb2a4065c3cfd', run_name='xgboost', run_uuid='81

In [25]:
runs= client.search_runs(
    experiment_ids='1',
    filter_string="", #se puede usar un tag en especial o algún parámetro de interés ej: 'tags.model = "lasso_regression" '
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse_valid ASC"]

)

In [26]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse_valid: {run.data.metrics['rmse_valid']}")

run id: 28e70ccddd654c95a8fccbc0a6e2f19c, rmse_valid: 5.141356376318939
run id: a90d6cc190014a21ba5a0dc4de13f94e, rmse_valid: 5.141458543615687
run id: 65ed233fd0054be8889a1155ed47d3e6, rmse_valid: 5.141656982943444
run id: 612db07ec0be41f3979ef7d5ded289d5, rmse_valid: 5.1426372995313
run id: 3097dfcd2144412080fdcd9f475cc5ab, rmse_valid: 5.142957409728757


### Promote a model

In [27]:
run_id = "a90d6cc190014a21ba5a0dc4de13f94e"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri, "regressor_model")

Registered model 'regressor_model' already exists. Creating a new version of this model...
Created version '2' of model 'regressor_model'.


<ModelVersion: aliases=[], creation_timestamp=1700326839214, current_stage='None', description=None, last_updated_timestamp=1700326839214, name='regressor_model', run_id='a90d6cc190014a21ba5a0dc4de13f94e', run_link=None, source='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1/a90d6cc190014a21ba5a0dc4de13f94e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [37]:
model_name = "regressor_model"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f" version: {version.version} , actual stage: {version._current_stage}")

 version: 1 , actual stage: Production
 version: 2 , actual stage: Staging


In [35]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Staging"
)

<ModelVersion: aliases=[], creation_timestamp=1700326839214, current_stage='Staging', description=None, last_updated_timestamp=1700327076159, name='regressor_model', run_id='a90d6cc190014a21ba5a0dc4de13f94e', run_link=None, source='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1/a90d6cc190014a21ba5a0dc4de13f94e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [36]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production"
)

<ModelVersion: aliases=[], creation_timestamp=1700326130228, current_stage='Production', description='', last_updated_timestamp=1700327122306, name='regressor_model', run_id='245ccb12106e4ea5b990d7bb1f706a1f', run_link='', source='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1/245ccb12106e4ea5b990d7bb1f706a1f/artifacts/xgboost_model', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

Cuando vayas a cambiar el staging de un modelo a otro estado, podemos también añadir descripciones: 

In [44]:
client.update_model_version(
    name=model_name, 
    version=2,
    description=f"The model version {2} was transitioned to Production on {datetime.today().date()}"
)

<ModelVersion: aliases=[], creation_timestamp=1700326839214, current_stage='Staging', description='The model version 2 was transitioned to Production on 2023-11-18', last_updated_timestamp=1700327739292, name='regressor_model', run_id='a90d6cc190014a21ba5a0dc4de13f94e', run_link=None, source='/Users/mdurango/Proyect/Mlops-platzi/tracking/mlruns/1/a90d6cc190014a21ba5a0dc4de13f94e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [53]:
type(test.get_label())

numpy.ndarray

In [51]:
test.get_label()

array([10.616667, 10.583333, 10.583333, ..., 31.      , 22.      ,
       15.      ], dtype=float32)

In [52]:
test.get_data()

<69392x5702 sparse matrix of type '<class 'numpy.float32'>'
	with 135461 stored elements in Compressed Sparse Row format>

### Testing a register model: 

De esta forma se deben hacer las pruebas con el pipeline de producción para asegurar que el modelo se comporta como se espera, además de validar la infraestructura y las pruebas que recién habíamos mencionado. 

In [62]:
# OJO: Es importante que recuerdes los formatos permitidos por mlflow load_model, como dataframes, numpy arrays, etc.
def testint_model_from_mlflow(model_name: str, stage:str, X_test: xgboost.core.DMatrix, Y_test: np.ndarray):
    """this function tests a model from mlflow
    Args:
        model_name (str): name of the model
        stage (str): stage of the model
        X_test (scipy.sparse._csr.csr_matrix): test data
        Y_test (scipy.sparse._csr.csr_matrix): test target
    Returns:
        float: rmse of the model
    
    """
    model_uri = f"models:/{model_name}/{stage}"
    model = mlflow.pyfunc.load_model(model_uri)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(Y_test, y_pred, squared=False)
    return {"rmse": rmse}

In [63]:
%time
testint_model_from_mlflow(model_name= "regressor_model", stage="Production", X_test=test, Y_test=test.get_label())


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


{'rmse': 5.354509}

#### Ventajas de MLflow:

* Gestión de Ciclo de Vida: Facilita el seguimiento de experimentos, versionado de modelos y reproducción de resultados.
* Interoperabilidad: Es compatible con múltiples frameworks de aprendizaje automático y se integra fácilmente en flujos de trabajo existentes.
* Abierto y Modular: Ofrece una arquitectura modular que permite la flexibilidad y personalización.
* Trazabilidad y Reproducibilidad: Registra métricas, parámetros y artefactos para reproducir modelos y resultados.
* Comunidad Activa: Amplia comunidad de usuarios y contribuciones continuas.


#### Desventajas de MLflow:
* Complejidad para Grandes Volúmenes de Datos: Puede enfrentar dificultades al manejar grandes volúmenes de datos o flujos de trabajo muy complejos.
* Curva de Aprendizaje: Requiere tiempo para familiarizarse con todas sus funcionalidades y componentes.
Limitaciones en Algunas Funcionalidades: Algunas funcionalidades pueden no ser tan avanzadas o flexibles como en otras herramientas especializadas.


### Alternativas a MLflow:
* TensorBoard: Enfoque específico para TensorFlow, útil para visualizar gráficamente métricas, grafos de modelos y más.
* DVC (Data Version Control): Se enfoca en versionado de datos y modelos, y gestión de experimentos.
* Comet.ml: Ofrece seguimiento de experimentos, colaboración y visualización de manera similar a MLflow.
* Weights & Biases: Ofrece seguimiento de experimentos, colaboración y visualización de manera similar a MLflow.


#### Conclusiones

* Podemos trackear metadata

* Registrar modelos

* Obtener los requirimientos del ambiente de desarrollo donde fue entrenado los modelos

* Podemos hacer un seguimiento de los modelos y compararlos de forma fácil y amigable con la interfaz de MLflow y en código

* Hacer transiciones de estados de los modelos

* Añadir anotaciones o descripciones 

So, ya sabes incluír el registro de modelos en nuestro pipeline de ML, ahora vamos a aprender de workflows y tasks. ¡Te espero en la próxima clase! :) 